In [1]:
import numpy as np
from tenpy.networks.mps import TransferMatrix

In [3]:
def pollmann_turner(self, Op = 'inversion', charge='all'):
    """ Obtains the matrices of the action of a symmetry 'Op' on the right Schmidt states:
    
        Op_R |i; R> = U_{i, j} |j; R>
        
    Op is either: 
        - list of single site matrices to be applied to the sites
        - 'inversion' : inverts orbital ordering
        - 'time_reversal': complex conjugation
        - an MPS representing Op psi
    
    Returns:
        Tr(U U^*)/chi   (pm 1 for inv and time reversal)
        U
        q, the charge sector of dominant eigenvalue
    """
    self.convert_form('B')
    if Op=='inversion':
        psi_inv = self.copy().spatial_inversion() #Form spatial reflection of the WF
    elif Op=='time_reversal':
        psi_inv = self.copy()
        for i in range(psi_inv.L):
            psi_inv.setB(i, psi_inv.getB(i).iconj() )
     :
            psi_inv.setB(i, npc.tensordot(self.get(Op, i), psi_inv.getB(i), axes = [[1], [0]] ) )
            
            
    T = TransferMatrix(psi_inv, self) #Represents generalized T matrixBy
    
    
    #T does diagonalization by charge sector, so we must loop over sectors
    if charge =='all':
        q_sectors = T.qt_ind_dict.keys()
    else:
        q_sectors = charge
        
    eta_max = 0	#largest eta (eigenvalue) we've found so far
    
    for charge_sector in q_sectors:
        try:
            eta, G = T(charge_sector).eigenvectors(num_ev=1, verbose=False) #Find dominant eigenvector
            print("Q, |eta|", charge_sector, np.abs(eta))
        except:
            print("Sector", charge_sector, "did not converge.")
            
            raise
            #eta = [0.]
            #G = [None]
            
            #If this is getting to be a problem, we can do dense diag as below.
            #m = matvec_to_array(T(charge_sector))
            #eta, G =  np.linalg.eig(m)
            #perm = np.argsort(-np.abs(eta))
            #eta = eta[perm]
            #print eta
            #G = [ T(charge_sector).to_npc_array(G[:, 0]) ]
        
        #eta = eta[0] #eigenvalue
        #G = G[0] #This should be the 'U_I/chi' object of Pollmann-Turner
        
        if np.abs(eta) > np.abs(eta_max): #Or we could just check if it is 1...?
            if np.abs(eta_max - 1.) < 10**(-8):
                print("WARNING: T-mat has degeneracy at eta = 1.")
            U = G[0]
            eta_max = eta
            q_max = charge_sector
            #print G.shape[0]*npc.tensordot(G, G.conj(), [[1], [1]]) #This should be identity
            #print "eta, ph, G:", eta, ph, G
            
    U = U*np.sqrt(U.shape[0])
    print("Using charge sector", q_max, "|eta| = ", np.abs(eta_max))
    print("| U.Ud - 1|", npc.norm(npc.tensordot(U, U.conj(), axes = [[1], [1]]) - npc.eye_like(U)))
    s = self.s[-1]
    print("|Us - sU|", npc.norm(U.scale_axis(s, 0) - U.scale_axis(s, 1)))
    p = npc.inner(U, U.conj(), [[0, 1], [1, 0]])/U.shape[0]
            

    return p, U, q_max

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 24)

# Generate an MPS

In [1]:
# import the model file we designed
"""
    Here we want to clarify that the 'kitaev_ladder_snake1.py' file
    is different from the 'kitaev_ladder_snake.py' in that the *1 file 
    uses the original representation of xyz,
    i.e. Jz stands for the sigmazsigmaz interactions which starts from
    the first pair of neighbors; Jx being the second pair; and Jy being
    the third-nearest-neighbors.
    
    The purpose of designing so many different 1D correspondences is 
    to serve for different phases.
    For example, when the Jx interaction is very weak, the optimal 
    choice is to make Jx as the third-nearest neighbors; when Jy is 
    weak the similar case holds.
"""
from kitaev_ladder_snake1 import run_atomic, save_after_run, finite_scaling, load_data, fDMRG_KL

# the folder name for storing data
folder_prefix = 'snake/'

# generate the function for running and storing the data in the folder we want
run_save = save_after_run(run_atomic, folder_prefix=folder_prefix)

In [126]:
Jx = 1.75
Jy = 0.25
Jz = 1.0

In [143]:
chi_list = [2**k for k in range(3, 8)]
initial_psi = None

for chi in chi_list:
    result = run_atomic(Jx=Jx, Jy=Jy, Jz=Jz, chi=chi, initial_psi=initial_psi, L=2, bc='periodic', bc_MPS='infinite')
    psi = result['psi']
    initial_psi = psi.copy()
    pass

psi = initial_psi.copy()

Reading 'L'=2 for config KitaevLadderSnakeCompactModel
Reading 'conserve'=None for config KitaevLadderSnakeCompactModel
Reading 'S'=0.5 for config KitaevLadderSnakeCompactModel
Reading 'bc'='periodic' for config KitaevLadderSnakeCompactModel
Reading 'bc_MPS'='infinite' for config KitaevLadderSnakeCompactModel
Reading 'Jx'=1.75 for config KitaevLadderSnakeCompactModel
Reading 'Jy'=0.25 for config KitaevLadderSnakeCompactModel
Reading 'Jz'=1.0 for config KitaevLadderSnakeCompactModel


==============================START==============================
Chi =  8 

Subconfig 'trunc_params'=Config(<3 options>, 'trunc_params') for config TwoSiteDMRGEngine
Initial sweeps...
Reading 'start_env'=10 for config TwoSiteDMRGEngine
Updating environment
..........
Reading 'N_sweeps_check'=10 for config TwoSiteDMRGEngine
Reading 'max_sweeps'=200 for config TwoSiteDMRGEngine
Reading 'max_E_err'=1e-06 for config TwoSiteDMRGEngine
Reading 'max_S_err'=0.0001 for config TwoSiteDMRGEngine
Reading 'mixer'=True

/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/tools/params.py:225: UserWarning: unused option ['P_tol'] for config lanczos_params

  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


....
Reading 'N_sweeps_check'=10 for config TwoSiteDMRGEngine
Reading 'max_sweeps'=200 for config TwoSiteDMRGEngine
Reading 'max_E_err'=1e-06 for config TwoSiteDMRGEngine
Reading 'max_S_err'=0.0001 for config TwoSiteDMRGEngine
Reading 'mixer'=True for config TwoSiteDMRGEngine
Subconfig 'mixer_params'=Config(<4 options>, 'mixer_params') for config TwoSiteDMRGEngine
Updating environment
.....
sweep 10, age = 204
Energy = -0.9586462705795178, S = nan, norm_err = 1.1e-08
Current memory usage 294.2 MB, time elapsed: 2.7 s
Delta E = nan, Delta S = 0.0000e+00 (per sweep)
max_trunc_err = 1.6679e-09, max_E_trunc = 1.6935e-08
MPS bond dimensions: [16, 16, 16, 16]
Updating environment
.....
sweep 20, age = 324
Energy = -0.9586464289910950, S = nan, norm_err = 3.3e-09
Current memory usage 294.2 MB, time elapsed: 4.6 s
Delta E = -1.5841e-08, Delta S = 0.0000e+00 (per sweep)
max_trunc_err = 1.2065e-09, max_E_trunc = 6.5274e-09
MPS bond dimensions: [16, 16, 16, 16]
Convergence criterium reached with 

/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2334: UserWarning: canonical_form_infinite: project to smaller bond dimension
  Gl, Wr = self._canonical_form_correct_left(i1, Gl, Wr)
/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2355: UserWarning: canonical_form_infinite: project to smaller bond dimension
  Gl, Wr = self._canonical_form_correct_left(j1, Gl, Wr_list[j1 % L])


.........
Reading 'N_sweeps_check'=10 for config TwoSiteDMRGEngine
Reading 'max_sweeps'=200 for config TwoSiteDMRGEngine
Reading 'max_E_err'=1e-06 for config TwoSiteDMRGEngine
Reading 'max_S_err'=0.0001 for config TwoSiteDMRGEngine
Reading 'mixer'=True for config TwoSiteDMRGEngine
Subconfig 'mixer_params'=Config(<4 options>, 'mixer_params') for config TwoSiteDMRGEngine
Updating environment
.....
sweep 10, age = 204
Energy = -0.9586464333186129, S = nan, norm_err = 7.2e-12
Current memory usage 294.2 MB, time elapsed: 3.0 s
Delta E = nan, Delta S = 0.0000e+00 (per sweep)
max_trunc_err = 7.7185e-18, max_E_trunc = 2.8422e-13
MPS bond dimensions: [54, 58, 35, 56]
Updating environment
.....
sweep 20, age = 324
Energy = -0.9586464333186342, S = nan, norm_err = 3.5e-13
Current memory usage 294.2 MB, time elapsed: 7.1 s
Delta E = -2.1316e-15, Delta S = 0.0000e+00 (per sweep)
max_trunc_err = 5.7345e-18, max_E_trunc = 5.1159e-13
MPS bond dimensions: [48, 58, 34, 59]
Convergence criterium reached 

/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2334: UserWarning: canonical_form_infinite: project to smaller bond dimension
  Gl, Wr = self._canonical_form_correct_left(i1, Gl, Wr)
/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2179: UserWarning: canonical_form_infinite: project to smaller bond dimension
  self.canonical_form_infinite(renormalize)
/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2355: UserWarning: canonical_form_infinite: project to smaller bond dimension
  Gl, Wr = self._canonical_form_correct_left(j1, Gl, Wr_list[j1 % L])


.........
Reading 'N_sweeps_check'=10 for config TwoSiteDMRGEngine
Reading 'max_sweeps'=200 for config TwoSiteDMRGEngine
Reading 'max_E_err'=1e-06 for config TwoSiteDMRGEngine
Reading 'max_S_err'=0.0001 for config TwoSiteDMRGEngine
Reading 'mixer'=True for config TwoSiteDMRGEngine
Subconfig 'mixer_params'=Config(<4 options>, 'mixer_params') for config TwoSiteDMRGEngine
Updating environment
.....
sweep 10, age = 204
Energy = -0.9586464333186662, S = nan, norm_err = 7.0e-12
Current memory usage 294.2 MB, time elapsed: 3.2 s
Delta E = nan, Delta S = 0.0000e+00 (per sweep)
max_trunc_err = 1.3280e-19, max_E_trunc = 1.1369e-13
MPS bond dimensions: [52, 57, 36, 57]
Updating environment
.....
sweep 20, age = 324
Energy = -0.9586464333186129, S = nan, norm_err = 1.7e-13
Current memory usage 294.2 MB, time elapsed: 8.8 s
Delta E = 5.3291e-15, Delta S = 0.0000e+00 (per sweep)
max_trunc_err = 1.6276e-19, max_E_trunc = 5.6843e-14
MPS bond dimensions: [47, 57, 35, 57]
Convergence criterium reached w

/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2334: UserWarning: canonical_form_infinite: project to smaller bond dimension
  Gl, Wr = self._canonical_form_correct_left(i1, Gl, Wr)
/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2179: UserWarning: canonical_form_infinite: project to smaller bond dimension
  self.canonical_form_infinite(renormalize)
/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2355: UserWarning: canonical_form_infinite: project to smaller bond dimension
  Gl, Wr = self._canonical_form_correct_left(j1, Gl, Wr_list[j1 % L])


In [149]:
# Apply the local operators `Sx` of each site
psi_test = psi.copy()

op = 'Sigmax'

for i in range(psi_test.L):
    psi_test.apply_local_op(i, op)
    
TM = TransferMatrix(psi, psi_test)
eta, G = TM.eigenvectors(num_ev=1)
U = G[0]
Ux = U.split_legs()
Ux *= np.sqrt(Ux.shape[0])
print(op, ' eta = ', eta)

# Apply the local operators `Sy` of each site
psi_test = psi.copy()

op = 'Sigmay'

for i in range(psi_test.L):
    psi_test.apply_local_op(i, op)
    
TM = TransferMatrix(psi, psi_test)
eta, G = TM.eigenvectors(num_ev=1)
U = G[0]
Uy = U.split_legs()
Uy *= np.sqrt(Uy.shape[0])
print(op, ' eta = ', eta)

chi=(Ux.shape[0])
a = 1
b = 0
c = 1
d = 0

Uxy = npc.tensordot(Ux, Uy, axes=([a], [b]))
Uyx = npc.tensordot(Uy, Ux, axes=([c], [d]))
print(npc.inner(Uxy, Uyx.conj()) / chi)

Sigmax  eta =  [1.+4.90718499e-17j]
Sigmay  eta =  [1.-8.48028862e-18j]
(-1.000712600645315+6.344131569286608e-17j)


/home/jerry/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: FutureWarning: inner(): `axes` currently defaults to 'range', will change to 'labels'


In [153]:
Jx = 1.75
Jy = 0.25
Jz = 1.0

In [154]:
chi_list = [2**k for k in range(3, 8)]
initial_psi = None

for chi in chi_list:
    result = run_atomic(Jx=Jx, Jy=Jy, Jz=Jz, chi=chi, initial_psi=initial_psi, L=4, bc='periodic', bc_MPS='infinite')
    psi = result['psi']
    initial_psi = psi.copy()
    pass

psi = initial_psi.copy()

Reading 'L'=4 for config KitaevLadderSnakeCompactModel
Reading 'conserve'=None for config KitaevLadderSnakeCompactModel
Reading 'S'=0.5 for config KitaevLadderSnakeCompactModel
Reading 'bc'='periodic' for config KitaevLadderSnakeCompactModel
Reading 'bc_MPS'='infinite' for config KitaevLadderSnakeCompactModel
Reading 'Jx'=1.75 for config KitaevLadderSnakeCompactModel
Reading 'Jy'=0.25 for config KitaevLadderSnakeCompactModel
Reading 'Jz'=1.0 for config KitaevLadderSnakeCompactModel


==============================START==============================
Chi =  8 

Subconfig 'trunc_params'=Config(<3 options>, 'trunc_params') for config TwoSiteDMRGEngine
Initial sweeps...
Reading 'start_env'=10 for config TwoSiteDMRGEngine
Updating environment
..........
Reading 'N_sweeps_check'=10 for config TwoSiteDMRGEngine
Reading 'max_sweeps'=200 for config TwoSiteDMRGEngine
Reading 'max_E_err'=1e-06 for config TwoSiteDMRGEngine
Reading 'max_S_err'=0.0001 for config TwoSiteDMRGEngine
Reading 'mixer'=True

/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/tools/params.py:225: UserWarning: unused option ['P_tol'] for config lanczos_params

  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


.......
Reading 'N_sweeps_check'=10 for config TwoSiteDMRGEngine
Reading 'max_sweeps'=200 for config TwoSiteDMRGEngine
Reading 'max_E_err'=1e-06 for config TwoSiteDMRGEngine
Reading 'max_S_err'=0.0001 for config TwoSiteDMRGEngine
Reading 'mixer'=True for config TwoSiteDMRGEngine
Subconfig 'mixer_params'=Config(<4 options>, 'mixer_params') for config TwoSiteDMRGEngine
Updating environment
.....
sweep 10, age = 408
Energy = -0.9586463480121949, S = nan, norm_err = 9.7e-09
Current memory usage 294.2 MB, time elapsed: 5.3 s
Delta E = nan, Delta S = 0.0000e+00 (per sweep)
max_trunc_err = 1.6686e-09, max_E_trunc = 1.6943e-08
MPS bond dimensions: [16, 16, 16, 16, 16, 16, 16, 16]
Updating environment
.....
sweep 20, age = 648
Energy = -0.9586464291770582, S = nan, norm_err = 2.9e-09
Current memory usage 294.2 MB, time elapsed: 8.6 s
Delta E = -8.1165e-09, Delta S = 0.0000e+00 (per sweep)
max_trunc_err = 1.2026e-09, max_E_trunc = 6.5095e-09
MPS bond dimensions: [16, 16, 16, 16, 16, 16, 16, 16]


/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2334: UserWarning: canonical_form_infinite: project to smaller bond dimension
  Gl, Wr = self._canonical_form_correct_left(i1, Gl, Wr)
/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2355: UserWarning: canonical_form_infinite: project to smaller bond dimension
  Gl, Wr = self._canonical_form_correct_left(j1, Gl, Wr_list[j1 % L])


Before the canonicalization:
Bond dim =  [28, 32, 26, 32, 26, 32, 26, 32]
Canonicalizing...
The norm is:  1.0
The overlap is:  (1.0000000000000089-8.965642154920766e-17j)
After the canonicalization:
Bond dim =  [28, 32, 26, 32, 26, 32, 26, 32]
Computing properties
Optimizing
Loop for chi=32 done.
============================== END ==============================
Reading 'L'=4 for config KitaevLadderSnakeCompactModel
Reading 'conserve'=None for config KitaevLadderSnakeCompactModel
Reading 'S'=0.5 for config KitaevLadderSnakeCompactModel
Reading 'bc'='periodic' for config KitaevLadderSnakeCompactModel
Reading 'bc_MPS'='infinite' for config KitaevLadderSnakeCompactModel
Reading 'Jx'=1.75 for config KitaevLadderSnakeCompactModel
Reading 'Jy'=0.25 for config KitaevLadderSnakeCompactModel
Reading 'Jz'=1.0 for config KitaevLadderSnakeCompactModel


==============================START==============================
Chi =  64 

Subconfig 'trunc_params'=Config(<3 options>, 'trunc_params') for conf

/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2334: UserWarning: canonical_form_infinite: project to smaller bond dimension
  Gl, Wr = self._canonical_form_correct_left(i1, Gl, Wr)
/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2179: UserWarning: canonical_form_infinite: project to smaller bond dimension
  self.canonical_form_infinite(renormalize)
/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2355: UserWarning: canonical_form_infinite: project to smaller bond dimension
  Gl, Wr = self._canonical_form_correct_left(j1, Gl, Wr_list[j1 % L])


Before the canonicalization:
Bond dim =  [28, 36, 28, 36, 28, 36, 28, 36]
Canonicalizing...
The norm is:  1.0
The overlap is:  (1.0000000000000058-1.6708306678214835e-16j)
After the canonicalization:
Bond dim =  [28, 36, 28, 36, 28, 36, 28, 36]
Computing properties
Optimizing
Loop for chi=64 done.
============================== END ==============================
Reading 'L'=4 for config KitaevLadderSnakeCompactModel
Reading 'conserve'=None for config KitaevLadderSnakeCompactModel
Reading 'S'=0.5 for config KitaevLadderSnakeCompactModel
Reading 'bc'='periodic' for config KitaevLadderSnakeCompactModel
Reading 'bc_MPS'='infinite' for config KitaevLadderSnakeCompactModel
Reading 'Jx'=1.75 for config KitaevLadderSnakeCompactModel
Reading 'Jy'=0.25 for config KitaevLadderSnakeCompactModel
Reading 'Jz'=1.0 for config KitaevLadderSnakeCompactModel


==============================START==============================
Chi =  128 

Subconfig 'trunc_params'=Config(<3 options>, 'trunc_params') for co

/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2334: UserWarning: canonical_form_infinite: project to smaller bond dimension
  Gl, Wr = self._canonical_form_correct_left(i1, Gl, Wr)
/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2179: UserWarning: canonical_form_infinite: project to smaller bond dimension
  self.canonical_form_infinite(renormalize)
/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2355: UserWarning: canonical_form_infinite: project to smaller bond dimension
  Gl, Wr = self._canonical_form_correct_left(j1, Gl, Wr_list[j1 % L])


In [155]:
# Apply the local operators `Sx` of each site
psi_test = psi.copy()

op = 'Sigmax'

for i in range(psi_test.L):
    psi_test.apply_local_op(i, op)
    
TM = TransferMatrix(psi, psi_test)
eta, G = TM.eigenvectors(num_ev=1)
U = G[0]
Ux = U.split_legs()
Ux *= np.sqrt(Ux.shape[0])
print(op, ' eta = ', eta)

# Apply the local operators `Sy` of each site
psi_test = psi.copy()

op = 'Sigmay'

for i in range(psi_test.L):
    psi_test.apply_local_op(i, op)
    
TM = TransferMatrix(psi, psi_test)
eta, G = TM.eigenvectors(num_ev=1)
U = G[0]
Uy = U.split_legs()
Uy *= np.sqrt(Uy.shape[0])
print(op, ' eta = ', eta)

chi=(Ux.shape[0])
a = 1
b = 0
c = 1
d = 0

Uxy = npc.tensordot(Ux, Uy, axes=([a], [b]))
Uyx = npc.tensordot(Uy, Ux, axes=([c], [d]))
print(npc.inner(Uxy, Uyx.conj()) / chi)

Sigmax  eta =  [1.-9.97449711e-17j]
Sigmay  eta =  [1.+5.73084686e-17j]
(-1.0007245107028977+0j)


/home/jerry/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: FutureWarning: inner(): `axes` currently defaults to 'range', will change to 'labels'


In [150]:
Jx = 0.25
Jy = 1.75
Jz = 1.0

In [151]:
chi_list = [2**k for k in range(3, 8)]
initial_psi = None

for chi in chi_list:
    result = run_atomic(Jx=Jx, Jy=Jy, Jz=Jz, chi=chi, initial_psi=initial_psi, L=2, bc='periodic', bc_MPS='infinite')
    psi = result['psi']
    initial_psi = psi.copy()
    pass

psi = initial_psi.copy()

Reading 'L'=2 for config KitaevLadderSnakeCompactModel
Reading 'conserve'=None for config KitaevLadderSnakeCompactModel
Reading 'S'=0.5 for config KitaevLadderSnakeCompactModel
Reading 'bc'='periodic' for config KitaevLadderSnakeCompactModel
Reading 'bc_MPS'='infinite' for config KitaevLadderSnakeCompactModel
Reading 'Jx'=0.25 for config KitaevLadderSnakeCompactModel
Reading 'Jy'=1.75 for config KitaevLadderSnakeCompactModel
Reading 'Jz'=1.0 for config KitaevLadderSnakeCompactModel


==============================START==============================
Chi =  8 

Subconfig 'trunc_params'=Config(<3 options>, 'trunc_params') for config TwoSiteDMRGEngine
Initial sweeps...
Reading 'start_env'=10 for config TwoSiteDMRGEngine
Updating environment
..........
Reading 'N_sweeps_check'=10 for config TwoSiteDMRGEngine
Reading 'max_sweeps'=200 for config TwoSiteDMRGEngine
Reading 'max_E_err'=1e-06 for config TwoSiteDMRGEngine
Reading 'max_S_err'=0.0001 for config TwoSiteDMRGEngine
Reading 'mixer'=True

/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2179: UserWarning: canonical_form_infinite: project to smaller bond dimension
  self.canonical_form_infinite(renormalize)
/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2328: UserWarning: Although we renormalized the TransferMatrix, the largest eigenvalue is not 1
  warnings.warn("Although we renormalized the TransferMatrix, "
/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/tools/params.py:225: UserWarning: unused option ['P_tol'] for config lanczos_params

  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


........
Reading 'N_sweeps_check'=10 for config TwoSiteDMRGEngine
Reading 'max_sweeps'=200 for config TwoSiteDMRGEngine
Reading 'max_E_err'=1e-06 for config TwoSiteDMRGEngine
Reading 'max_S_err'=0.0001 for config TwoSiteDMRGEngine
Reading 'mixer'=True for config TwoSiteDMRGEngine
Subconfig 'mixer_params'=Config(<4 options>, 'mixer_params') for config TwoSiteDMRGEngine
Updating environment
.....
sweep 10, age = 204
Energy = -0.9586464248626712, S = nan, norm_err = 4.8e-05
Current memory usage 294.2 MB, time elapsed: 3.5 s
Delta E = nan, Delta S = 0.0000e+00 (per sweep)
max_trunc_err = 2.0838e-09, max_E_trunc = 1.4638e-08
MPS bond dimensions: [16, 16, 16, 16]
Updating environment
.....
sweep 20, age = 324
Energy = -0.9586464278885174, S = nan, norm_err = 1.3e-08
Current memory usage 294.2 MB, time elapsed: 5.4 s
Delta E = -3.0258e-10, Delta S = 0.0000e+00 (per sweep)
max_trunc_err = 1.8497e-09, max_E_trunc = 1.3943e-08
MPS bond dimensions: [16, 16, 16, 16]
Convergence criterium reached w

/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2334: UserWarning: canonical_form_infinite: project to smaller bond dimension
  Gl, Wr = self._canonical_form_correct_left(i1, Gl, Wr)
/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2355: UserWarning: canonical_form_infinite: project to smaller bond dimension
  Gl, Wr = self._canonical_form_correct_left(j1, Gl, Wr_list[j1 % L])


........
Reading 'N_sweeps_check'=10 for config TwoSiteDMRGEngine
Reading 'max_sweeps'=200 for config TwoSiteDMRGEngine
Reading 'max_E_err'=1e-06 for config TwoSiteDMRGEngine
Reading 'max_S_err'=0.0001 for config TwoSiteDMRGEngine
Reading 'mixer'=True for config TwoSiteDMRGEngine
Subconfig 'mixer_params'=Config(<4 options>, 'mixer_params') for config TwoSiteDMRGEngine
Updating environment
.....
sweep 10, age = 204
Energy = -0.9586464333186981, S = nan, norm_err = 1.5e-11
Current memory usage 294.2 MB, time elapsed: 3.3 s
Delta E = nan, Delta S = 0.0000e+00 (per sweep)
max_trunc_err = 1.7372e-17, max_E_trunc = 1.4211e-13
MPS bond dimensions: [54, 61, 36, 62]
Updating environment
.....
sweep 20, age = 324
Energy = -0.9586464333187905, S = nan, norm_err = 1.8e-12
Current memory usage 294.2 MB, time elapsed: 7.6 s
Delta E = -9.2371e-15, Delta S = 0.0000e+00 (per sweep)
max_trunc_err = 6.9649e-18, max_E_trunc = 2.2737e-13
MPS bond dimensions: [47, 60, 34, 59]
Convergence criterium reached w

/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2334: UserWarning: canonical_form_infinite: project to smaller bond dimension
  Gl, Wr = self._canonical_form_correct_left(i1, Gl, Wr)
/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2179: UserWarning: canonical_form_infinite: project to smaller bond dimension
  self.canonical_form_infinite(renormalize)
/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2355: UserWarning: canonical_form_infinite: project to smaller bond dimension
  Gl, Wr = self._canonical_form_correct_left(j1, Gl, Wr_list[j1 % L])


Before the canonicalization:
Bond dim =  [28, 36, 28, 36]
Canonicalizing...
The norm is:  1.0
The overlap is:  (1.0000000000000069-2.352622660214646e-16j)
After the canonicalization:
Bond dim =  [28, 36, 28, 36]
Computing properties
Optimizing
Loop for chi=64 done.
============================== END ==============================
Reading 'L'=2 for config KitaevLadderSnakeCompactModel
Reading 'conserve'=None for config KitaevLadderSnakeCompactModel
Reading 'S'=0.5 for config KitaevLadderSnakeCompactModel
Reading 'bc'='periodic' for config KitaevLadderSnakeCompactModel
Reading 'bc_MPS'='infinite' for config KitaevLadderSnakeCompactModel
Reading 'Jx'=0.25 for config KitaevLadderSnakeCompactModel
Reading 'Jy'=1.75 for config KitaevLadderSnakeCompactModel
Reading 'Jz'=1.0 for config KitaevLadderSnakeCompactModel


==============================START==============================
Chi =  128 

Subconfig 'trunc_params'=Config(<3 options>, 'trunc_params') for config TwoSiteDMRGEngine
Initial sw

/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2334: UserWarning: canonical_form_infinite: project to smaller bond dimension
  Gl, Wr = self._canonical_form_correct_left(i1, Gl, Wr)
/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2179: UserWarning: canonical_form_infinite: project to smaller bond dimension
  self.canonical_form_infinite(renormalize)
/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2355: UserWarning: canonical_form_infinite: project to smaller bond dimension
  Gl, Wr = self._canonical_form_correct_left(j1, Gl, Wr_list[j1 % L])


In [152]:
# Apply the local operators `Sx` of each site
psi_test = psi.copy()

op = 'Sigmax'

for i in range(psi_test.L):
    psi_test.apply_local_op(i, op)
    
TM = TransferMatrix(psi, psi_test)
eta, G = TM.eigenvectors(num_ev=1)
U = G[0]
Ux = U.split_legs()
Ux *= np.sqrt(Ux.shape[0])
print(op, ' eta = ', eta)

# Apply the local operators `Sy` of each site
psi_test = psi.copy()

op = 'Sigmay'

for i in range(psi_test.L):
    psi_test.apply_local_op(i, op)
    
TM = TransferMatrix(psi, psi_test)
eta, G = TM.eigenvectors(num_ev=1)
U = G[0]
Uy = U.split_legs()
Uy *= np.sqrt(Uy.shape[0])
print(op, ' eta = ', eta)

chi=(Ux.shape[0])
a = 1
b = 0
c = 1
d = 0

Uxy = npc.tensordot(Ux, Uy, axes=([a], [b]))
Uyx = npc.tensordot(Uy, Ux, axes=([c], [d]))
print(npc.inner(Uxy, Uyx.conj()) / chi)

Sigmax  eta =  [1.+4.26234322e-17j]
Sigmay  eta =  [1.-5.06863218e-17j]
(-1.0006400756325249-3.172065784643304e-17j)


/home/jerry/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: FutureWarning: inner(): `axes` currently defaults to 'range', will change to 'labels'


In [133]:
Ux.shape

(28, 28)

In [134]:
U2 = npc.tensordot(Ux, Ux.conj(), ([1], [0])).to_ndarray()
U2 *= 28
U2

array([[ 2.83458102e+01+5.25432503e-02j, -5.50369047e-16+3.43844495e-01j,
        -1.83820425e-11+4.72269312e-11j, -2.55117743e-11-7.89894039e-11j,
        -4.82427238e-13-8.30059702e-11j,  1.49964683e-12-4.57556358e-11j,
        -1.63459137e-13+3.57921886e-13j,  2.68272920e-12-4.03271373e-13j,
         2.88771059e-12-9.79573794e-11j, -5.40254336e-12-6.02495629e-11j,
         2.19778275e-12+1.07506912e-12j, -1.40482255e-12-1.77297510e-13j,
        -6.20893164e-11-9.96502110e-12j, -1.98419690e-11+8.94450215e-11j,
        -8.64125534e-12+2.95872840e-12j,  5.76582664e-12-1.14496892e-11j,
         1.24216998e-11+6.34338216e-12j, -7.47445352e-12+2.15608312e-11j,
         4.64111825e-11-1.27488785e-11j,  1.77496248e-11+7.16258016e-11j,
        -1.07887038e-10+7.93136500e-11j, -9.29006689e-11-1.21022084e-10j,
        -9.30207582e-11+5.31202831e-11j,  2.27950681e-11+6.60593309e-12j,
         1.17257480e-11-1.26220520e-11j,  3.15518675e-11+6.25509829e-12j,
        -4.81764742e-11-8.78528578e-11

In [104]:
TM.pipe

LegPipe([LegCharge(ChargeInfo([], []), qconj=+1,
array([ 0, 28]), array([], shape=(1, 0), dtype=int64)),
LegCharge(ChargeInfo([], []), qconj=-1,
array([ 0, 28]), array([], shape=(1, 0), dtype=int64))],
qconj=+1, sort=True, bunch=True)

Sigmay  eta =  [1.-6.14473517e-17j]


In [136]:
Uy.shape

(28, 28)

In [137]:
npc.inner(Ux, Ux.conj())

/home/jerry/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: inner(): `axes` currently defaults to 'range', will change to 'labels'
  """Entry point for launching an IPython kernel.


(27.999999999999993+0j)

In [138]:
npc.inner(Uy, Uy.conj())

/home/jerry/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: inner(): `axes` currently defaults to 'range', will change to 'labels'
  """Entry point for launching an IPython kernel.


(28.000000000000007+0j)

(-1.0007126006453146+0j)


/home/jerry/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: inner(): `axes` currently defaults to 'range', will change to 'labels'
  if __name__ == '__main__':


In [61]:
import numpy as np
import tenpy.linalg.np_conserved as npc

In [62]:
eta, G = TM.eigenvectors(num_ev=1)
U = G[0]
U = U.split_legs()
print("| U.Ud - 1| = ", npc.norm(npc.tensordot(U, U.conj(), axes = [[1], [1]]) - npc.eye_like(U)))
print(npc.tensordot(U, U.conj(), axes = [[1], [1]]) - npc.eye_like(U))

| U.Ud - 1| =  5.1025228739735695
<npc.Array shape=(28, 28) charge=ChargeInfo([], []) labels=['vL', 'vL*']
 +1  | -1  
0  []|0  []
28   |28   
>


In [49]:
G

[<npc.Array shape=(784,) charge=ChargeInfo([], []) labels=['(vL.vL*)']>]

In [45]:
U.legs

[LegPipe([LegCharge(ChargeInfo([], []), qconj=+1,
 array([ 0, 28]), array([], shape=(1, 0), dtype=int64)),
 LegCharge(ChargeInfo([], []), qconj=-1,
 array([ 0, 28]), array([], shape=(1, 0), dtype=int64))],
 qconj=+1, sort=True, bunch=True)]

In [42]:
npc.eye_like(U)

<npc.Array shape=(784, 784) charge=ChargeInfo([], []) labels=[None, None]>

In [38]:
U = U*np.sqrt(U.shape[0])
print("| U.Ud - 1|", npc.norm(npc.tensordot(U, U.conj(), axes = [[1], [1]]) - npc.eye_like(U)))
s = self.s[-1]
print("|Us - sU|", npc.norm(U.scale_axis(s, 0) - U.scale_axis(s, 1)))
p = npc.inner(U, U.conj(), [[0, 1], [1, 0]])/U.shape[0]

IndexError: list index out of range

In [48]:
import tenpy
tenpy.__version__

'0.7.1'